In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import lit
from pyspark.sql.functions import substring

In [ ]:
os.getcwd()

In [ ]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

In [ ]:
# Read CSV file
df = spark.read.csv("data/members_v3.csv", header=True, inferSchema=True)

In [ ]:
# Extract year and month
dfm_with_ym = df.withColumn("year", substring("registration_init_time", 1, 4)) \
                .withColumn("month", substring("registration_init_time", 5, 2))

# Count distinct years and months
dfm_with_ym.select("year").distinct().count(), dfm_with_ym.select("month").distinct().count()

In [ ]:
dfm_with_ym.write \
    .mode("overwrite") \
    .option("header", "true") \
    .parquet("datamart/bronze/members")

In [ ]:
spark.stop()